In [1]:
!pip3 install -U sentence-transformers
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Import Library

In [2]:
import pandas as pd
import numpy as np
import re
import string

from nltk.tokenize import ToktokTokenizer
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
import heapq
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


toktok = ToktokTokenizer()
lemma = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))



In [3]:
jobs_df = pd.read_csv('JobsData.csv')
jobs_df

,Job Name,Company Name,JD,Skills,Date Posted,YOE,Location,Website,Job Function:,Industry:,Specialization:,Qualification:,Hiring Location:,Role:,Vacancies:
0,Python Engineer,east india securities ltd.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",Posted 5 days ago,2 - 5 yrs,Kolkata,http://www.eisec.com/,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,NaN,NaN,NaN
1,PYTHON DEVELOPER,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",Posted 6 days ago,4 - 7 yrs,Bengaluru / Bangalore,NaN,IT Software : Software Products & Services,"Telecom, IT-Hardware/Networking",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN,NaN,NaN
2,Python Developer,InnOvator Web Solutions Pvt.Ltd.,Job Category: DevelopmentJob Type: Full TimeJo...,"rest,python,django,webdeveloper,mysql,api",Posted 6 days ago,5 - 8 yrs,Mumbai,http://www.innovatorwebsolutions.com,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,NaN,NaN,NaN
3,Python - Odoo,AxisTechnolabs,Responsibilities : We are looking for Freshers...,"python,django,itskills,html5,api,jquery",Posted 6 days ago,0 - 1 yrs,Ahmedabad,http://www.axistechnolabs.com,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN,NaN,NaN
4,Python Developer,pearl global solutions,"Python Developer Django Job Type : Full time,...","python,database,django,teamplayer,sql",Posted 6 days ago,4 - 7 yrs,Cochin/ Kochi/ Ernakulam,https://pearlglobalsolutions.com/,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,Planning and Performance Management Analyst,Armaco Chemical Processes Systems Pvt Ltd,Description: Aramco energizes the world econom...,"reports,managementaccounting,reporting,cfa,exc...",Posted few days ago,1 - 3 yrs,Jeddah,http://www.armacocps.com/,Accounting/Tax/Company Secretary/Audit,"Manufacturing/Industrial, Petrochemicals",Bookkeeping,MBA/ PGDM\r\n \r\n ...,NaN,NaN,NaN
2246,Cluster Manager - SALPL - Prime,Bajaj Finserv Lending Ltd,Cluster Manager - SALPL - Prime Location HYDER...,"msexcel,unicaandsalesforce,communication,motiv...",Posted few days ago,2 - 5 yrs,Hyderabad/Secunderabad,http://www.bajajfinserv.in/finance/,IT Software : Software Products & Services,Financial Services/Stockbroking,Software Engineer,ME/ M.Tech./ MS (Engg/ Sciences)\r\n ...,NaN,NaN,NaN
2247,Workforce Analyst,MSNU Recruiters\r\n (More Jobs),"Hello all,\r\nOne of the best opportunity for ...","DataAnalysis,teamleader,projectmanager,project...",Posted 5 days ago,1 - 4 yrs,(More Jobs),NaN,Operations/Customer Service/Telecalling/Backend,"Consulting Services, Internet/Dot com/ISP",Non-Voice Process - Domestic BPO Type,Any Graduate,NaN,"Data Entry/ DTP/ Data Conversion - Operator, D...",NaN
2248,Accounting Trainee | D A K S M And Company LLP,Anitej Services Pvt Ltd,"About the job Looking for smart, vibrant and e...","msexcel,tally,accounting,internship,finance,ta...",Posted few days ago,0 - 3 yrs,Noida/Greater Noida,http://www.cos.youth4work.com,Accounting/Tax/Company Secretary/Audit,IT-Software,ICWA,ME/ M.Tech./ MS (Engg/ Sciences)\r\n ...,NaN,NaN,NaN


# dataset Preprocessing


In [4]:
jobs_df.shape

(2250, 15)

In [5]:
jobs_df.columns

Index(['Job Name', 'Company Name', 'JD', 'Skills', 'Date Posted', 'YOE',
       'Location', 'Website', 'Job Function:', 'Industry:', 'Specialization:',
       'Qualification:', 'Hiring Location:', 'Role:', 'Vacancies:'],
      dtype='object')

In [6]:
jobs_df.isnull().sum()

Job Name               0
Company Name           0
JD                     0
Skills                 0
Date Posted            0
YOE                    0
Location              71
Website              296
Job Function:          0
Industry:              0
Specialization:        0
Qualification:         0
Hiring Location:    2244
Role:               2163
Vacancies:          2238
dtype: int64

In [7]:
jobs_df=jobs_df.fillna(' ')

In [8]:
jobs_df.isnull().sum()

Job Name            0
Company Name        0
JD                  0
Skills              0
Date Posted         0
YOE                 0
Location            0
Website             0
Job Function:       0
Industry:           0
Specialization:     0
Qualification:      0
Hiring Location:    0
Role:               0
Vacancies:          0
dtype: int64

In [9]:
jobs_df['Job Name']=jobs_df['Job Name']+['Specialization:']

In [10]:
# Drop multiple columns
columns_to_drop = ['Vacancies:', 'Role:', 'Hiring Location:', 'Qualification:','Industry:','Specialization:','Job Function:','Website','Location','Date Posted']
jobs_df = jobs_df.drop(columns=columns_to_drop, axis=1)


In [11]:
jobs_df.head()

,Job Name,Company Name,JD,Skills,YOE
0,Python EngineerSpecialization:,east india securities ltd.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",2 - 5 yrs
1,PYTHON DEVELOPERSpecialization:,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",4 - 7 yrs
2,Python DeveloperSpecialization:,InnOvator Web Solutions Pvt.Ltd.,Job Category: DevelopmentJob Type: Full TimeJo...,"rest,python,django,webdeveloper,mysql,api",5 - 8 yrs
3,Python - OdooSpecialization:,AxisTechnolabs,Responsibilities : We are looking for Freshers...,"python,django,itskills,html5,api,jquery",0 - 1 yrs
4,Python DeveloperSpecialization:,pearl global solutions,"Python Developer Django Job Type : Full time,...","python,database,django,teamplayer,sql",4 - 7 yrs


In [12]:
jobs_df['full_text'] = jobs_df['Job Name'] + ' ' + jobs_df['JD'] + ' ' + jobs_df['Company Name'] + ' ' + jobs_df['Skills']
jobs_df['raw_full_text'] = jobs_df['Job Name'] + ' ' + jobs_df['JD'] + ' ' + jobs_df['Company Name'] + ' ' + jobs_df['Skills']


In [13]:
df = jobs_df[['full_text', 'raw_full_text']].copy()


# Text preprocessing

In [14]:
# products preprocessing
#1-Replaces specific HTML entities like "br", or empty strings.
#2-Uses a regular expression to remove HTML tags (text enclosed in '<' and '>').
#3-Replaces consecutive whitespaces with a single space.
def cleanhtml(raw_html):
    '''
    Input: HTML Text
    Output: Cleaned HTML Text
    <tag> </tag> removed and other cleaning
    '''
    cleantext = ''
    try:
        raw_html = raw_html.replace(" br ", " ")
        raw_html = raw_html.replace("nbsp"," ")
        raw_html = raw_html.replace("ndash"," ")
        raw_html = raw_html.replace("&rsquo;", ' ')
        raw_html = raw_html.replace("&trade;", ' ')
        raw_html = raw_html.replace("&amp",' ')
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, ' ', raw_html)
        cleantext = ' '.join(cleantext.split())
    except:
        print(raw_html)
    return cleantext


In [15]:
#cleaning and preparing text data
#1- convert the text to lowercase
#2-removes punctuation from the txet 
#3- split the text into a list of word 
#4-Joins the list of words back into a single string with words separated by a single space.
def text_preprocessing(text):
    return ' '.join(text.lower().translate(str.maketrans('', '', string.punctuation)).split())

df['full_text'] = df['full_text'].apply(text_preprocessing)


In [16]:
df['full_text'][2]

'python developerspecialization job category developmentjob type full timejob location malad wwe are looking for a python web developer responsible for managing the interchange of data between the server and the usersgood to have a knowledge of django web frameworkknowledge of python django framework understanding of serverside scripting languagesstrong experience using web services and apis in pythonhands on python django experience esp with mysql django rest frameworkminimum 5 years experience in designing and developing applications using python innovator web solutions pvtltd restpythondjangowebdevelopermysqlapi'

In [17]:
def _lemma(text):
    try:
        tokens = [lemma.lemmatize(token) for token in toktok.tokenize(text) if token not in stop_words]
        lemmatized_text = ' '.join(tokens)
    except:
        print(text)
    return lemmatized_text

df['full_text'] = np.vectorize(_lemma)(df['full_text'])


In [18]:
df

,full_text,raw_full_text
0,python engineerspecialization jobdescription 2...,Python EngineerSpecialization: job_description...
1,python developerspecialization python develope...,PYTHON DEVELOPERSpecialization: PYTHON DEVELOP...
2,python developerspecialization job category de...,Python DeveloperSpecialization: Job Category: ...
3,python odoospecialization responsibility looki...,Python - OdooSpecialization: Responsibilities ...
4,python developerspecialization python develope...,Python DeveloperSpecialization: Python Develop...
...,...,...
2245,planning performance management analystspecial...,Planning and Performance Management AnalystSpe...
2246,cluster manager salpl primespecialization clus...,Cluster Manager - SALPL - PrimeSpecialization:...
2247,workforce analystspecialization hello one best...,"Workforce AnalystSpecialization: Hello all,\r\..."
2248,accounting trainee k company llpspecialization...,Accounting Trainee | D A K S M And Company LLP...


# Iteration1 :  TF-IDF



#Lexical Similarity Model
 
#Based on Syntactic Similarity


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf = TfidfVectorizer(
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 2),
    stop_words='english',#Removes common English stop words
    analyzer='word' #Determines whether to use word-level analysis.
)
#Use fit_transform to transform the 'full_text' column into a matrix of TF-IDF features.
# Fit and transform the 'full_text' column using TfidfVectorizer
features = tfidf.fit_transform(df['full_text'].values.tolist())

# Create a DataFrame with TF-IDF features
df_tfidf = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())


In [37]:
similarity_matrix = cosine_similarity(df_tfidf, df_tfidf)


In [38]:
similarity_matrix.shape

(2250, 2250)

In [39]:
# Replace the similarity_matrix with your actual similarity matrix, 
#and modify the given_item_index accordingly based on the item for which you want to find similar items.
# top 10 similar items to an given item
heapq.nlargest(10, enumerate(similarity_matrix[0]), key=lambda x: x[1])

[(0, 0.9999999999999999),
 (2015, 0.0994293993837945),
 (123, 0.08557878965001171),
 (191, 0.0726670362405347),
 (68, 0.06840485226836315),
 (192, 0.0648278882867852),
 (106, 0.060211663655823325),
 (501, 0.05774068807765704),
 (1461, 0.05772577411897258),
 (1457, 0.055965949696678806)]

In [50]:
import heapq

df['recommendation_list_tfidf'] = df.index.map(
    lambda indx: [a for a, b in heapq.nlargest(
        10, enumerate(similarity_matrix[indx]), key=lambda x: x[1]
    )][1:]
)


In [51]:
df

,full_text,raw_full_text,recommendation_list_tfidf
0,python engineerspecialization jobdescription 2...,Python EngineerSpecialization: job_description...,"[2015, 123, 191, 68, 192, 106, 501, 1461, 1457]"
1,python developerspecialization python develope...,PYTHON DEVELOPERSpecialization: PYTHON DEVELOP...,"[2, 29, 12, 55, 7, 88, 67, 70, 199]"
2,python developerspecialization job category de...,Python DeveloperSpecialization: Job Category: ...,"[70, 88, 126, 199, 24, 25, 1, 113, 204]"
3,python odoospecialization responsibility looki...,Python - OdooSpecialization: Responsibilities ...,"[15, 1886, 818, 1115, 94, 180, 776, 1477, 645]"
4,python developerspecialization python develope...,Python DeveloperSpecialization: Python Develop...,"[2, 199, 70, 88, 104, 105, 12, 23, 18]"
...,...,...,...
2245,planning performance management analystspecial...,Planning and Performance Management AnalystSpe...,"[2026, 2231, 2105, 2127, 2104, 2227, 2114, 210..."
2246,cluster manager salpl primespecialization clus...,Cluster Manager - SALPL - PrimeSpecialization:...,"[2249, 1111, 1112, 1113, 498, 499, 2108, 1748,..."
2247,workforce analystspecialization hello one best...,"Workforce AnalystSpecialization: Hello all,\r\...","[2234, 2156, 460, 461, 462, 463, 464, 468, 469]"
2248,accounting trainee k company llpspecialization...,Accounting Trainee | D A K S M And Company LLP...,"[1007, 818, 1115, 180, 776, 645, 1477, 496, 94]"
